### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [265]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [266]:
#calculate total number of schools
school_count = school_data["school_name"].count()

#calculate total number of students
student_sum = school_data_complete["student_name"].count()

#calculate total budget
school_budget = school_data["budget"].sum()

#calcualte average scores
math_mean = student_data["math_score"].mean()
read_mean = student_data["reading_score"].mean()

In [267]:
#create the bins in which Data will be held 
bins = [0, 69.9, 100]

#create the names for the bins
group_names = ["<70", "70+"]

#math and read pass add to df
school_data_complete["Math_Pass"]= pd.cut(school_data_complete["math_score"], bins, labels=group_names, include_lowest=True)
school_data_complete["Read_Pass"]= pd.cut(school_data_complete["reading_score"], bins, labels=group_names, include_lowest=True)

math70 = school_data_complete.loc[school_data_complete["Math_Pass"] == "70+"]
math70_count = math70["Math_Pass"].count()
math70_percent = (math70_count / student_sum)*100

read70 = school_data_complete.loc[school_data_complete["Read_Pass"] == "70+"]
read70_count = read70["Read_Pass"].count()
read70_percent = (read70_count / student_sum)*100


In [268]:
#calculate % that passed both
both70 = math70.loc[school_data_complete["Read_Pass"] == "70+"]
both70_count = both70["Math_Pass"].count()
both70_percent = (both70_count / student_sum)*100

In [269]:
# initialize list of lists
q1data = [[school_count,student_sum,school_budget,math_mean,read_mean,math70_percent,read70_percent,both70_percent]]
  
# Create the pandas DataFrame
q1_df = pd.DataFrame(q1data, 
                        columns = ['Total Schools','Total Students','Total Budget',
                                   'Average Math Score','Average Reading Score','% Passing Math',
                                   '% Passing Reading','% Passing Both']
                       )
  
# print dataframe.
q1_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [270]:
#add per student budget to df
budget_per=school_data['budget']/school_data['size']
school_data['Per Student Budget']=budget_per


In [271]:
#group by to get math and reading means
schoolread_mean = school_data_complete.groupby(['school_name'])['reading_score'].mean().reset_index()
schoolmath_mean = school_data_complete.groupby(['school_name'])['math_score'].mean().reset_index()
studentcount = school_data_complete.groupby(['school_name'])['student_name'].count().reset_index()

In [272]:
#calculate % passing scores
schoolread70_count = read70.groupby(['school_name'])['Read_Pass'].count().reset_index()
schoolmath70_count = math70.groupby(['school_name'])['Math_Pass'].count().reset_index()
schoolboth70_count = both70.groupby(['school_name'])['student_name'].count().reset_index()

In [273]:
#merge item dataframes
q2addread = pd.merge(school_data, schoolread70_count, how="left", on=["school_name", "school_name"])
q2addmath = pd.merge(q2addread, schoolmath70_count, how="left", on=["school_name", "school_name"])
q2addboth = pd.merge(q2addmath, schoolboth70_count, how="left", on=["school_name", "school_name"])
q2addread_mean = pd.merge(q2addboth, schoolread_mean, how="left", on=["school_name", "school_name"])
q2all = pd.merge(q2addread_mean, schoolmath_mean, how="left", on=["school_name", "school_name"]) 

In [276]:
#calculate percents
q2mathpercent = (q2all['Math_Pass']/q2all['size'])*100
q2readpercent = (q2all['Read_Pass']/q2all['size'])*100
q2bothpercent = (q2all['student_name']/q2all['size'])*100

#add in percents to df
q2all['% Passing Math']=q2mathpercent
q2all['% Passing Reading']=q2readpercent
q2all['% Overall Passing']=q2bothpercent

In [277]:
#drop columns
q2alldrop = q2all.drop(['School ID','Read_Pass','Math_Pass','student_name'],axis=1)

#rename and clean
q2final = q2alldrop.rename(columns={"school_name":"School Name","type":"School Type",
                              "size":"Total Students","budget":"Total School Budget",
                              "reading_score":"Average Reading Score","math_score":"Average Math Score"})
q2final

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,65.988471,80.739234,53.204476
2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,93.867121,95.854628,89.892107
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,66.752967,80.862999,53.527508
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.392371,97.138965,90.599455
5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.867718,96.539641,90.582567
6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,94.133477,97.039828,91.334769
7,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,66.680064,81.933280,54.642283
8,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,92.505855,96.252927,89.227166
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [278]:
#sort for top five
q2final = q2final.sort_values("% Overall Passing", ascending=False)
q2final.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,94.133477,97.039828,91.334769
14,Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,93.272171,97.308869,90.948012
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.392371,97.138965,90.599455
5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.867718,96.539641,90.582567
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [279]:
#sort for top five
q2final = q2final.sort_values("% Overall Passing", ascending=True)
q2final.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
11,Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,66.366592,80.220055,52.988247
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,65.988471,80.739234,53.204476
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,66.752967,80.862999,53.527508
12,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [280]:
#calculations for each grade
nineth = school_data_complete.loc[school_data_complete["grade"] == "9th"]
nineth_mean = nineth.groupby(['school_name'])['math_score'].mean().reset_index()
nineth_mean = nineth_mean.rename(columns={"math_score":"9th"})

tenth = school_data_complete.loc[school_data_complete["grade"] == "10th"]
tenth_mean = tenth.groupby(['school_name'])['math_score'].mean().reset_index()
tenth_mean = tenth_mean.rename(columns={"math_score":"10th"})

eleventh = school_data_complete.loc[school_data_complete["grade"] == "11th"]
eleventh_mean = eleventh.groupby(['school_name'])['math_score'].mean().reset_index()
eleventh_mean = eleventh_mean.rename(columns={"math_score":"11th"})

twelth = school_data_complete.loc[school_data_complete["grade"] == "12th"]
twelth_mean = twelth.groupby(['school_name'])['math_score'].mean().reset_index()
twelth_mean = twelth_mean.rename(columns={"math_score":"12th"})

In [281]:
#merge tables
merge1 = pd.merge(nineth_mean, tenth_mean, how="left", on=["school_name", "school_name"])
merge2 = pd.merge(merge1, eleventh_mean, how="left", on=["school_name", "school_name"])
math_total = pd.merge(merge2, twelth_mean, how="left", on=["school_name", "school_name"])
math_total

,school_name,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [282]:
#calculations for each grade
ninethread = school_data_complete.loc[school_data_complete["grade"] == "9th"]
ninethread_mean = ninethread.groupby(['school_name'])['reading_score'].mean().reset_index()
ninethread_mean = ninethread_mean.rename(columns={"reading_score":"9th"})

tenthread = school_data_complete.loc[school_data_complete["grade"] == "10th"]
tenthread_mean = tenthread.groupby(['school_name'])['reading_score'].mean().reset_index()
tenthread_mean = tenthread_mean.rename(columns={"reading_score":"10th"})

eleventhread = school_data_complete.loc[school_data_complete["grade"] == "11th"]
eleventhread_mean = eleventhread.groupby(['school_name'])['reading_score'].mean().reset_index()
eleventhread_mean = eleventhread_mean.rename(columns={"reading_score":"11th"})

twelthread = school_data_complete.loc[school_data_complete["grade"] == "12th"]
twelthread_mean = twelthread.groupby(['school_name'])['reading_score'].mean().reset_index()
twelthread_mean = twelthread_mean.rename(columns={"reading_score":"12th"})

In [283]:
#merge tables
merge3 = pd.merge(ninethread_mean, tenthread_mean, how="left", on=["school_name", "school_name"])
merge4 = pd.merge(merge3, eleventhread_mean, how="left", on=["school_name", "school_name"])
read_total = pd.merge(merge4, twelthread_mean, how="left", on=["school_name", "school_name"])
read_total

,school_name,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [284]:
#create the bins in which Data will be held 
bins = [0, 584.9, 629.9, 644.9, 900]

#create the names for the bins
group_names = ["<$585", "$585-630", "$630-645", "$645-800"]

#add bins to df
q2all["Spending Ranges (Per Student)"]= pd.cut(q2all["Per Student Budget"], bins, labels=group_names, include_lowest=True)

#calculate averages
budget_readmean = q2all.groupby(['Spending Ranges (Per Student)'])['reading_score'].mean().reset_index()
budget_mathmean = q2all.groupby(['Spending Ranges (Per Student)'])['math_score'].mean().reset_index()
budget_readpass = q2all.groupby(['Spending Ranges (Per Student)'])['Read_Pass'].sum().reset_index()
budget_mathpass = q2all.groupby(['Spending Ranges (Per Student)'])['Math_Pass'].sum().reset_index()
budget_bothpass = q2all.groupby(['Spending Ranges (Per Student)'])['student_name'].sum().reset_index()
budget_students = q2all.groupby(['Spending Ranges (Per Student)'])['size'].sum().reset_index()

#add in percents to df
budgetmerge = pd.merge(budget_mathmean, budget_readmean, how="left", on=["Spending Ranges (Per Student)", "Spending Ranges (Per Student)"])
budgetmerge1 = pd.merge(budgetmerge, budget_readpass, how="left",  on=["Spending Ranges (Per Student)", "Spending Ranges (Per Student)"])
budgetmerge2 = pd.merge(budgetmerge1, budget_mathpass, how="left",  on=["Spending Ranges (Per Student)", "Spending Ranges (Per Student)"])
budgetmerge3 = pd.merge(budgetmerge2, budget_students, how="left",  on=["Spending Ranges (Per Student)", "Spending Ranges (Per Student)"])
budgetall = pd.merge(budgetmerge3, budget_bothpass, how="left",  on=["Spending Ranges (Per Student)", "Spending Ranges (Per Student)"])

#calculate percents
budgetmathpercent = (budgetall['Math_Pass']/budgetall['size'])*100
budgetreadpercent = (budgetall['Read_Pass']/budgetall['size'])*100
budgetallpercent = (budgetmathpercent+budgetreadpercent)/2

#add percent data
budgetall['% Passing Math']=budgetmathpercent
budgetall['% Passing Reading']=budgetreadpercent
budgetall['% Overall Passing']=budgetallpercent

#drop columns
budgetdrop = budgetall.drop(['Read_Pass','Math_Pass','student_name','size'],axis=1)

#rename and clean
budgetfinal = budgetdrop.rename(columns={"math_score":"Average Math Score","reading_score":"Average Reading Score"})
budgetfinal['% Passing Math'] = budgetfinal['% Passing Math'].map("{:.2f}%".format)
budgetfinal['% Passing Reading'] = budgetfinal['% Passing Reading'].map("{:.2f}%".format)
budgetfinal['% Overall Passing'] = budgetfinal['% Overall Passing'].map("{:.2f}%".format)

budgetfinal


,Spending Ranges (Per Student),Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,<$585,83.455399,83.933814,93.70%,96.69%,95.19%
1,$585-630,81.899826,83.155286,79.11%,88.51%,83.81%
2,$630-645,78.518855,81.624473,70.62%,82.60%,76.61%
3,$645-800,76.997210,81.027843,66.23%,81.11%,73.67%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [285]:
#create the bins in which Data will be held 
bins = [0, 999.9, 1999.9, 9000]

#create the names for the bins
group_names = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]

#add bins to df
q2all["School Size"]= pd.cut(q2all["size"], bins, labels=group_names, include_lowest=True)

#calculate averages
size_readmean = q2all.groupby(['School Size'])['reading_score'].mean().reset_index()
size_mathmean = q2all.groupby(['School Size'])['math_score'].mean().reset_index()
size_readpass = q2all.groupby(['School Size'])['Read_Pass'].sum().reset_index()
size_mathpass = q2all.groupby(['School Size'])['Math_Pass'].sum().reset_index()
size_bothpass = q2all.groupby(['School Size'])['student_name'].sum().reset_index()
size_students = q2all.groupby(['School Size'])['size'].sum().reset_index()

#add in percents to df
sizemerge = pd.merge(size_mathmean, size_readmean, how="left", on=["School Size", "School Size"])
sizemerge1 = pd.merge(sizemerge, size_readpass, how="left", on=["School Size", "School Size"])
sizemerge2 = pd.merge(sizemerge1, size_mathpass, how="left", on=["School Size", "School Size"])
sizemerge3 = pd.merge(sizemerge2, size_students, how="left", on=["School Size", "School Size"])
sizeall = pd.merge(sizemerge3, size_bothpass, how="left", on=["School Size", "School Size"])

#calculate percents
sizemathpercent = (sizeall['Math_Pass']/sizeall['size'])*100
sizereadpercent = (sizeall['Read_Pass']/sizeall['size'])*100
sizeallpercent = (sizemathpercent+sizereadpercent)/2

#add percent data
sizeall['% Passing Math']=sizemathpercent
sizeall['% Passing Reading']=sizereadpercent
sizeall['% Overall Passing']=sizeallpercent

#drop columns
sizedrop = sizeall.drop(['Read_Pass','Math_Pass','student_name','size'],axis=1)

#rename and clean
sizefinal = sizedrop.rename(columns={"math_score":"Average Math Score","reading_score":"Average Reading Score"})
sizefinal['% Passing Math'] = sizefinal['% Passing Math'].map("{:.2f}%".format)
sizefinal['% Passing Reading'] = sizefinal['% Passing Reading'].map("{:.2f}%".format)
sizefinal['% Overall Passing'] = sizefinal['% Overall Passing'].map("{:.2f}%".format)

sizefinal

,School Size,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Small(<1000),83.821598,83.929843,93.95%,96.04%,95.00%
1,Medium(1000-2000),83.374684,83.864438,93.62%,96.77%,95.19%
2,Large(2000-5000),77.746417,81.344493,68.65%,82.13%,75.39%


## Scores by School Type

* Perform the same operations as above, based on school type

In [286]:
#base mean and count
type_readmean = q2all.groupby(['type'])['reading_score'].mean().reset_index()
type_mathmean = q2all.groupby(['type'])['math_score'].mean().reset_index()
type_readpass = q2all.groupby(['type'])['Read_Pass'].sum().reset_index()
type_mathpass = q2all.groupby(['type'])['Math_Pass'].sum().reset_index()
type_bothpass = q2all.groupby(['type'])['student_name'].sum().reset_index()
type_students = q2all.groupby(['type'])['size'].sum().reset_index()

#merge data
typemerge = pd.merge(type_mathmean, type_readmean, how="left", on=["type", "type"])
typemerge1 = pd.merge(typemerge, type_readpass, how="left", on=["type", "type"])
typemerge2 = pd.merge(typemerge1, type_mathpass, how="left", on=["type", "type"])
typemerge3 = pd.merge(typemerge2, type_students, how="left", on=["type", "type"])
typeall = pd.merge(typemerge3, type_bothpass, how="left", on=["type", "type"])

#calculate percents
q2mathpercent = (typeall['Math_Pass']/typeall['size'])*100
q2readpercent = (typeall['Read_Pass']/typeall['size'])*100
q2allpercent = (q2mathpercent+q2readpercent)/2

#add percent data
typeall['% Passing Math']=q2mathpercent
typeall['% Passing Reading']=q2readpercent
typeall['% Overall Passing']=q2allpercent

#drop columns
typedrop = typeall.drop(['Read_Pass','Math_Pass','student_name','size'],axis=1)

#rename and clean
typefinal = typedrop.rename(columns={"math_score":"Average Math Score","reading_score":"Average Reading Score"})
typefinal['% Passing Math'] = typefinal['% Passing Math'].map("{:.2f}%".format)
typefinal['% Passing Reading'] = typefinal['% Passing Reading'].map("{:.2f}%".format)
typefinal['% Overall Passing'] = typefinal['% Overall Passing'].map("{:.2f}%".format)

typefinal


,type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Charter,83.473852,83.896421,93.70%,96.65%,95.17%
1,District,76.956733,80.966636,66.52%,80.91%,73.71%
